https://icasas101.github.io/FinalDSTutorial/

# Identifying Bias in the NOPD

By Josh Kellner and Isabella Casas

CMPS 3160 - Introduction to Data Science
Professor Mattei

## Introduction

For our Final Tutorial, we have partnered up to analyze this dataset called “Stop and Search (Field Interviews).” It is filled with data regarding instances of people being questioned by the New Orleans Police Department. Some of the information about these interviews includes when and where it happened, the officer conducting the questioning and potential search, a description of the individual being searched including age, gender, race, height and weight, the reason the interview was conducted, actions taken, etc. We plan to analyze this information in a way such that one can use our analysis to learn about any biases that NOPD has, or a lack thereof, and how these biases manifest themselves. We have not started this analysis yet but we expect to specifically look at relationships between frequencies of interviews and searches and descriptors of the subjects of these interviews and searches as well as the relationships between the severity of the actions taken by the police and the descriptors of the subjects. The dataset provides information about the car that the subject was driving, if they were driving one, which will be another variable that can shed light on biases. The issue of police brutality against black people in our country has, of course, been as widely discussed as ever over the last year because of the widespread abundance protests against it all over the country. This issue has by no means only been brought up around this time when many people are reckoning with it, but the ability of social media to spread information and set trends so quickly has given the issue a major spotlight. Our goal is to uncover the truth about many peoples’ suspicion of police which is that they are more likely to question, search, and take more severe actions against people of color, specifically black people. Given the other descriptions of the subjects besides race we will be able to identify any combinations characteristics that if present in a person or situation, could lead to an increase in the likelihood of being questioned, searched, etc. A completely random, hypothetical conclusion that could potentially be drawn from our analysis could show that women in a certain NOPD zone, which we could use to determine the neighborhood, are more likely to be pulled over than men between the hours of 7-9pm. In an ideal situation, any unfair biases that are discovered by this project will be used to determine a change in or create policy that would correct these unethical discriminatory actions and prompt city officials to put that policy in place. At the moment, we are focusing on New Orleans but a possible expansion, given that the datasets would be accessible, could be to compare the conclusions that we draw to conclusions drawn from other cities’ data. In terms of a collaboration plan, we have a Github repository set up to keep track of our most up to date work as well as each update. Every two weeks we plan to meet on Zoom to divide specific chunks of work to be done. In these meetings we will review the work we have done since the last meeting and work through things that we couldn’t complete individually, together.

## Data ETL

In [1]:
!head ../FinalDSTutorial/Stop_and_Search__Field_Interviews_.csv

'head' is not recognized as an internal or external command,
operable program or batch file.


In [2]:
import pandas as pd

In [3]:
pd.DataFrame

pandas.core.frame.DataFrame

Our data comes from: https://data.nola.gov/Public-Safety-and-Preparedness/Stop-and-Search-Field-Interviews-/kitu-f4uy/data

This dataset provides information on persons apprehended by the NOPD due to various reasons. This information includes the interviewees' race, age, vehicle make and model, and gender. 

In [12]:
df = pd.read_csv("../FinalDSTutorial/Stop_and_Search__Field_Interviews_.csv")
df.head()

C:\Users\Isabe\Documents\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3058: DtypeWarning: Columns (1,7,28) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,FieldInterviewID,NOPD_Item,EventDate,District,Zone,OfficerAssignment,StopDescription,ActionsTaken,VehicleYear,VehicleMake,...,SubjectWeight,SubjectEyeColor,SubjectHairColor,SubjectDriverLicState,CreatedDateTime,LastModifiedDateTime,Longitude,Latitude,Zip,BlockAddress
0,17415,NaN,01/01/2010 01:11:00 AM,6,E,6th District,TRAFFIC VIOLATION,NaN,2005.0,DODGE,...,160.0,Brown,Black,LA,01/01/2010 01:26:26 AM,NaN,0.0,0.0,NaN,NaN
1,17416,NaN,01/01/2010 02:06:00 AM,5,D,5th District,CALL FOR SERVICE,NaN,NaN,NaN,...,140.0,Brown,Black,NaN,01/01/2010 02:27:38 AM,NaN,0.0,0.0,NaN,NaN
2,17416,NaN,01/01/2010 02:06:00 AM,5,D,5th District,CALL FOR SERVICE,NaN,NaN,NaN,...,145.0,Brown,Black,NaN,01/01/2010 02:27:38 AM,NaN,0.0,0.0,NaN,NaN
3,17416,NaN,01/01/2010 02:06:00 AM,5,D,5th District,CALL FOR SERVICE,NaN,NaN,NaN,...,140.0,Brown,Black,NaN,01/01/2010 02:27:38 AM,NaN,0.0,0.0,NaN,NaN
4,17416,NaN,01/01/2010 02:06:00 AM,5,D,5th District,CALL FOR SERVICE,NaN,NaN,NaN,...,140.0,Brown,Black,NaN,01/01/2010 02:27:38 AM,NaN,0.0,0.0,NaN,NaN
